# "Single-nucleus Multiomic Atlas of Frontal Cortex in ALS: A Deep Learning Approach"

This Jupyter notebook is dedicated to reproducing results from the paper:

> "Single-nucleus Multiomic Atlas of Frontal Cortex in Amyotrophic Lateral Sclerosis with a Deep Learning-based Decoding of Alternative Polyadenylation Mechanisms" by Paul M. Mckeever and Aiden M. Sababi (2023).

The paper presents a comprehensive analysis of single-nucleus multiomics in the context of amyotrophic lateral sclerosis (ALS), focusing on alternative polyadenylation mechanisms using deep learning techniques.

The primary objective of this notebook is to replicate key analysis steps in the study, providing a transparent and reproducible path for validation and further exploration.

The paper can be found [here](https://www.biorxiv.org/content/10.1101/2023.12.22.573083v1.full).


In [ ]:
library(ggplot2)
library(dplyr)
library(stringr)
library(ggplot2)
library(tidyverse)
library(parallel)
library(MAAPER)

# MAAPER
> we use a package called [MAAPER](https://github.com/Vivianstats/MAAPER/) to confidently assing reads to the PA atlas.


In [ ]:
bam_root <- '/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/'
setwd(bam_root)

reference.gtf <- '/home/aiden/data/refgenome/refdata-gex-GRCh38-2020-A/genes//genes.gtf'
apa_atlas <- readRDS('/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/human.PAS.hg38.rds')
outdir <- "/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs"


call_mapper <- function(path, c1, c2){
    outdir <- "/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs"
    ast_map_out <- paste0(outdir,"/", path)
    maaper(reference.gtf, apa_atlas, bam_c1=c1, bam_c2=c2, 
          output_dir=ast_map_out, read_len=91, ncores=62, bed=TRUE)

}


## example of running MAAPER for pseudobulk celltypes. 
## It is recommended to run this individually for each celltype comparison, in case of memory issues and other errors.
setwd('../Microglia/')
bamfiles = list.files(pattern = "*.bam$")
control_bams <- as.vector(paste0(root,"Microglia/",bamfiles[which(startsWith(bamfiles, "CTRL"))]))
C9ALS_bams <- as.vector(paste0(root,"Microglia/",bamfiles[which(startsWith(bamfiles, "C9ALS"))]))
sALS_bams <- as.vector(paste0(root,"Microglia/",bamfiles[which(startsWith(bamfiles, "sALSnoFTLD"))]))

call_mapper("Microglia_C9ALSvsCTRL", C9ALS_bams, control_bams)

# APAlog
> we used the [APAlog](https://github.com/goodarzilab/APAlog) package to delve deeper into differential poly(A) site usage patterns. APAlog operates on the normalized counts of reads mapped to each poly(A) site to assess the extent and nature of differential usage. For a comprehensive comparison, APAlog was run in Pairwise Test mode, which enables the comparison of all possible pairs of poly(A) sites per transcript

In [ ]:
setwd('/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/')

In [ ]:
# functions
nmor_factor <- function (expression.data.frame, data.columns){
  gm_mean_z <- function(x){
    exp(sum(log(x)) / length(x))
  }
  edf <- expression.data.frame
  id.names <- names(edf)
  geo.mean.vec <- apply(edf[,data.columns], 1, function(x) gm_mean_z(x))
  ratios.df <- edf[,data.columns]/geo.mean.vec
  # Division by 0 gm_mean will create NAs here.
  normalization.factors <- apply(ratios.df, 2, function(x) median(x, na.rm=TRUE))
  return(normalization.factors)
}


## read in the table and then normalize and make count table for it
count_table_for_APAlog <- function(df){
    sf <- nmor_factor(df, c(8:9))
    df <- df %>% mutate(ALS = ALS_PAcount/sf[1], CTRL = ctrl_PAcount/sf[2])
    df <- df %>% gather(condition, normalized_count, `ALS`:`CTRL`) %>% arrange(pas, type, condition)
    return(df)
}
root_dir <- "/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/"
# sub_cts <- list.files()
sub_cts <- c('Astrocytes_C9ALSvsCTRL', 'Astrocytes_sALSvsCTRL') ## Sep 20. 2022>> doing this for only astrocytes
for (ct in sub_cts){
    inp_df <- read.csv(paste0(ct,"/pas_counts.txt"), sep='\t')
    out_file <- paste0(ct,"/APAlog_pas_count_input.txt")
    out_df <- count_table_for_APAlog(inp_df)
    write.table(out_df, file=out_file, quote=F, sep='\t', row.names = F)   
}

In [ ]:
## do this for for all the celltypes and conditions
run_APAlog <- function(ct) {
    
    root_dir <- "/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/"
    ## read in and process the count dataframe
    inp_df <- read.csv(paste0(ct,"/APAlog_pas_count_input.txt"), sep='\t')
    inp_df <- inp_df[, c(1,2,10,11)]
    colnames(inp_df) <- c('transcript', 'pA.site','sample','count')
    inp_df[sapply(inp_df, is.character)] <- lapply(inp_df[sapply(inp_df, is.character)], as.factor)
    
    ## make the design table
    design_table <- data.frame(sample=c('ALS', 'CTRL'),
                           Condition=c('ALS_pathology','Control'))
    design_table$sample <- factor(design_table$sample, levels=c('CTRL', 'ALS'))
    design_table$Condition <- factor(design_table$Condition, levels=c('Control', 'ALS_pathology'))
    
    ## run the APAlog in overal mode
    fit.o_test <- APAlog::pA_logit_dev(inp_df,
                                     pA.site ~ Condition,
                                     design_table,
                                     "sample",  
                                     adj_method = "fdr")
    ## run the APAlog in pairwise mode
    fit.p_test <- APAlog::pA_logit_pairwise(inp_df, pA.site~Condition, design_table, "sample")
    
    ## merge the dataframes
    fit.op_test <- merge(fit.o_test, fit.p_test, by = "transcript")
    
    outname <- paste0(root_dir,ct,"/APAlog_res.tsv")
    write.table(fit.op_test, file=outname, quote=F, sep='\t', row.names = F)
    
    plt <- volcano_plot_2(fit.op_test,
                     x='b_ConditionALS_pathology',
                     y='p_ConditionALS_pathology',
                     title=paste0('Volcano plot for ALS vs Control for ',ct),
                     pCutoff = 0.05,
                     FCcutoff = .5,
                     ylim=c(0,10), pointsize = 1)
    plot_name <- paste0(root_dir,ct,"/APAlog_volcano_plot.pdf")
    pdf(file = plot_name)
    print(plt)
    dev.off()
    
}


### adding needed extra informatio to the dataframes
add_meta_information <- function(ct){
    
    root_dir <- "/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/"
    inp_df <- read.csv(paste0(ct,"/APAlog_res.tsv"), sep='\t')
    inp_df <- inp_df %>% mutate(strand=gsub(".*:.*:(.*)","\\1",ref_site)) %>%
                mutate(multiplyer= ifelse(strand=="+",1,-1)) %>%
                mutate(LFC_PA_Usage=b_ConditionALS_pathology*multiplyer) %>% 
                mutate(negative_logFDR=-log(p_ConditionALS_pathology, base=10)) %>%
                mutate(switch_width=as.numeric(gsub(".*:(.*):.*","\\1",alt_site)) - as.numeric(gsub(".*:(.*):.*","\\1",ref_site))) %>%
                mutate(bed = paste0(gsub("(.*):.*:.*","\\1",ref_site), ',',
                                    gsub(".*:(.*):.*","\\1",ref_site), ',',
                                    gsub(".*:(.*):.*","\\1",alt_site)))
    
    ### correcting handful of cases where the proximal and distal positions are reversed in ref_site and Alt_sites

    inp_df <- inp_df %>% mutate(correction_multiplyer = ifelse(switch_width <= -1, -1, 1)) %>%
                         mutate(switch_width=switch_width*correction_multiplyer) %>%
                         mutate(LFC_PA_Usage=LFC_PA_Usage*correction_multiplyer) %>%
                         mutate(switch_name = ifelse(correction_multiplyer==-1, paste0(gsub("(.*):.*:.*","\\1",ref_site),':',
                                                     transcript,":",gsub(".*:(.*):.*","\\1",alt_site),
                                                     ':',gsub(".*:(.*):.*","\\1",ref_site),':',strand),
                                                     paste0(gsub("(.*):.*:.*","\\1",ref_site),':',
                                                     transcript,":",gsub(".*:(.*):.*","\\1",ref_site),
                                                     ':',gsub(".*:(.*):.*","\\1",alt_site),':',strand)))
    
    outname <- paste0(root_dir,ct,"/APAlog_res_metadata_added.tsv")
    print(outname)
    write.table(inp_df, file=outname, quote=F, sep='\t', row.names = F)
    
}

## make the bed files to get the sequences
## add upstream of the proximal sites
get_bed_files <- function(ct){
    inp_df <- read.csv(paste0(ct,"/APAlog_res_metadata_added.tsv"), sep='\t')
    inp_df <- inp_df %>% filter(switch_width >= 200) %>% filter(switch_width <= 15000)
    inp_df <- data.frame(inp_df['switch_name'],inp_df['strand'])
    
    inp_df <- inp_df %>% mutate(chr=gsub("(.*):.*:.*:.*:.*","\\1",switch_name)) %>%
                          mutate(start=as.character(ifelse(strand=='+',
                                                           as.numeric(gsub(".*:.*:(.*):.*:.*","\\1",switch_name)) - 1000,
                                                           gsub(".*:.*:(.*):.*:.*","\\1",switch_name)))) %>%
                          mutate(end=as.character(ifelse(strand=='-',
                                                           as.numeric(gsub(".*:.*:.*:(.*):.*","\\1",switch_name)) + 1000,
                                                           gsub(".*:.*:.*:(.*):.*","\\1",switch_name)))) 
    
    
    inp_df <- data.frame(inp_df['chr'],inp_df['start'],inp_df['end'],inp_df['switch_name'])
    outname <- paste0(root_dir,ct,"/switch_region.bed")
    write.table(inp_df,file=outname, sep='\t', quote=F, row.names = F, col.names = F)
}

adj_p <- function(x, pcols, adj_method){
    y <- x[, pcols, drop = FALSE]
    if (adj_method == "qvalue"){
        y <- apply(y, 2, function(t) qvalue::qvalue(t)$qvalues)
    } else {
        y <- apply(y, 2, function(t) stats::p.adjust(t, method = adj_method))
    }

    newnames <- paste0(adj_method, "_", colnames(y))
    z <- data.frame(x,y)
    colnames(z)[(NCOL(x)+1) : NCOL(z)] <- newnames
    return(z)
}

# this function finds the signficance threshold based on the adjusted p values
get_sig_threshold <- function(df){
    df <- df %>% arrange(fdr_p_ConditionALS_pathology)
    return(df[which(df$fdr_p_ConditionALS_pathology > 0.05),][1,]$p_ConditionALS_pathology)
}

In [ ]:
r <- mclapply(sub_cts, run_APAlog, mc.cores = 32)  
r <- mclapply(sub_cts, add_meta_information, mc.cores = 28) 
r <- mclapply(sub_cts, get_bed_files, mc.cores = 28)  

In [ ]:
data_root = "/data1/APA/Paul_ALS_Data/bams_in/subscelltype_bamfiles/Mapper_outs/"
# list directories that ends with 'C9ALSvsCTRL'
sub_cts <- list.dirs(data_root, recursive = F, full.names = F)
c9_als_sub_cts <- sub_cts[grep("C9ALSvsCTRL", sub_cts)]
s_als_sub_cts <- sub_cts[grep("sALSvsCTRL", sub_cts)]